In [7]:
# Environ
import scipy as scp
import tensorflow as tf
from scipy.stats import gamma
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
import random
import multiprocessing as mp
import psutil
import pickle
import os
import re

# Own
import ddm_data_simulation as ds
import cddm_data_simulation as cds
import kde_training_utilities as kde_util
import kde_class as kde
import boundary_functions as bf

In [ ]:
# DDM  
repeats = 1
my_means = np.zeros(repeats)
for i in range(repeats):
    out = cds.ddm_flexbound(v = 0, 
                            a = 0.96,
                            w = 0.5,
                            ndt = 0.5,
                            delta_t = 0.01, 
                            max_t = 20,
                            n_samples = 20000,
                            boundary_fun = bf.weibull_cdf,
                            boundary_multiplicative = False, 
                            boundary_params = {'a': 1, 'beta': 2})
                            #boundary_params = {"theta": 0.01})
    print(i)
    
    my_means[i] = np.mean(out[0][out[1] == 1])

In [ ]:
def bin_simulator_output(out = [0, 0],
                         bin_dt = 0.04,
                         n_bins = 0,
                         eps_correction = 1e-7, # min p for a bin
                         params = ['v', 'a', 'w', 'ndt']
                        ): # ['v', 'a', 'w', 'ndt', 'angle']

    # Generate bins
    if n_bins == 0:
        n_bins = int(out[2]['max_t'] / bin_dt)
        bins = np.linspace(0, out[2]['max_t'], n_bins)
    else:    
        bins = np.linspace(0, out[2]['max_t'], n_bins)
        bins = np.append(bins, [100])
        print(bins)
    counts = []
    cnt = 0
    counts = np.zeros( (n_bins, len(out[2]['possible_choices']) ) )
    counts_size = counts.shape[0] * counts.shape[1]
    
    for choice in out[2]['possible_choices']:
        counts[:, cnt] = np.histogram(out[0][out[1] == choice], bins = bins)[0] / out[2]['n_samples']
        cnt += 1
    
    # Apply correction for empty bins
    n_small = 0
    n_big = 0
    n_small = np.sum(counts < eps_correction)
    n_big = counts_size - n_small 
    
    if eps_correction > 0:
        counts[counts <= eps_correction] = eps_correction
        counts[counts > eps_correction] -= (eps_correction * (n_small / n_big))

    return ([out[2][param] for param in params], # features
            counts, # labels
            {'max_t': out[2]['max_t'], 
             'bin_dt': bin_dt, 
             'n_samples': out[2]['n_samples']} # meta data
           )

In [ ]:
#%%timeit -n 1 -r 5
a, b = bin_simulator_output(out = out)

In [ ]:
out = cds.ornstein_uhlenbeck(v = 0.21,
                             a = 1.5,
                             w = 0.5,
                             g = 1,
                             ndt = 0.92,
                             boundary_fun = bf.constant,
                             n_samples = 100000)

In [ ]:
binned_sims = bin_simulator_output(out = out,
                                   n_bins = 256,
                                   eps_correction = 1e-7,
                                   params = ['v', 'a', 'w', 'g', 'ndt'])

In [ ]:
plt.plot(binned_sims[1][:, 1])
plt.plot(binned_sims[1][:, 0])

In [ ]:
binned_sims[1][255, 1]

In [ ]:
files_ = os.listdir('/media/data_cifs/afengler/data/kde/ddm/base_simulations_20000')

In [ ]:
labels = np.zeros((250000, 500, 2))
features = np.zeros((250000, 3))

cnt = 0
i = 0
file_dim = 100
for file_ in files_[:1000]:
    if file_[:8] == 'ddm_flex':
        out = pickle.load(open('/media/data_cifs/afengler/data/kde/ddm/base_simulations_20000/' + file_, 'rb'))
        features[cnt], labels[cnt] = bin_simulator_output(out = out)
        if cnt % file_dim == 0:
            print(cnt)
            pickle.dump((labels[(i * file_dim):((i + 1) * file_dim)], features[(i * file_dim):((i + 1) * file_dim)]), open('/media/data_cifs/afengler/data/kde/ddm/base_simulations_20000_binned/dataset_' + str(i), 'wb'))
            i += 1
        cnt += 1


In [ ]:
files_[0:10]

In [ ]:
random.shuffle(files_)
files_[0:10]

In [ ]:
# FULL DDM  
repeats = 50
my_means = np.zeros(repeats)
for i in range(repeats):
    out = cds.full_ddm(v = 0, 
                       a = 0.96,
                       w = 0.5,
                       ndt = 0.5,
                       dw = 0.0,
                       sdv = 0.0,
                       dndt = 0.5,
                       delta_t = 0.01, 
                       max_t = 20,
                       n_samples = 10000,
                       boundary_fun = bf.constant,
                       boundary_multiplicative = True, 
                       boundary_params = {})
    print(i)
    
    my_means[i] = np.mean(out[0][out[1] == 1])

In [ ]:
plt.hist(my_means, density = True)

In [ ]:
np.mean(out[0][out[1] == 1])

In [ ]:
plt.hist(out[0] * out[1], bins = 50)

In [ ]:
int(50 / out[2]['delta_t'] + 1)

In [ ]:
# LCA 
repeats = 1
my_means = np.zeros(repeats)
for i in range(repeats):
    out = cds.lca(v = np.array([0, 0], dtype = np.float32), 
                  a = 2, 
                  w = np.array([0.5, 0.5], dtype = np.float32), 
                  ndt = np.array([1.0, 1.0], dtype = np.float32),
                  g = -1.0,
                  b = 1.0,
                  delta_t = 0.01, 
                  max_t = 40,
                  n_samples = 10000,
                  boundary_fun = bf.constant,
                  boundary_multiplicative = True, 
                  boundary_params = {})
    print(i)
    my_means[i] = np.mean(out[0][out[1] == 1])

In [ ]:
out[1][out[1] == 0] = -1
plt.hist(out[0] * out[1], bins = 50)

In [ ]:
# LCA 
repeats = 10
my_means = np.zeros(repeats)
for i in range(repeats):
    out = cds.ddm_flexbound(v = 0.0, 
                            a = 1.5, 
                            w = 0.5, 
                            ndt = 0.1,
                            delta_t = 0.01, 
                            max_t = 40,
                            n_samples = 10000,
                            boundary_fun = bf.constant,
                            boundary_multiplicative = True, 
                            boundary_params = {})
    print(i)
    my_means[i] = np.mean(out[0][out[1] == 1])

In [ ]:
plt.hist(my_means, density = True)

In [ ]:
out[1][out[1] == -1].shape

In [ ]:
int((40 / 0.01) + 1)

In [ ]:
my_bound = bf.constant

In [ ]:
my_bound

In [ ]:
my_bound(t = 100)

In [ ]:
h_Test = (1, 1, 1)

In [ ]:
h_Test + (1,2)

In [ ]:
h = ({'c': 1}, {'c2': 2})

In [ ]:
h_Test + h

In [ ]:
a = ({'c': 1},)

In [ ]:
h_Test + a

In [ ]:
def foo(name, *args, **kwargs):
    print ("args: ", args)
    print ("Type of args: ", type(args))
    if len(args)>2:
        args = args[0], args[1]     #- Created Same name variable.
    print ("Temp args:", args)

In [ ]:
foo('hello', ('a', 'b', 'c'))

In [ ]:
t = ('a', 'b', 'c')

In [ ]:
t[0]

In [ ]:
t2 = t[0], t[1]

In [ ]:
t2

In [ ]:
a = ['a']
a = a + ['b', 'c']

In [ ]:
a

In [ ]:
test_dat = pd.DataFrame(np.zeros((10, 5)), columns = ['v_0', 'v_1', 'a', 'b', 'c'])

In [ ]:
test_dat

In [ ]:
np.array(test_dat.loc[1, ['v_0', 'v_1']])

In [ ]:
my_keys = []
for key in test_dat.keys():
    if key[0] == 'v':
        my_keys.append(key)
np.array(test_dat.loc[1, ['v_0', 'v_1']])

In [ ]:
my_target_folder = '/media/data_cifs/afengler/tmp/'

In [ ]:
import make_data_lba as mdlba

In [ ]:
my_dat = mdlba.make_data_rt_choice(target_folder = my_target_folder)

In [ ]:
a = [1,2,3,4,5]

In [ ]:
a[:-2]

In [ ]:
np.isclose(0, 1e-15)

In [ ]:
1.0 - 1e-29 == 1.0

In [ ]:
np.max(my_dat['log_likelihood'])

In [ ]:
data = np.concatenate([out[0], out[1]], axis = 1)

In [ ]:
data[data[:,0] < 3, 0]

In [ ]:
np.log(1e-100)

In [ ]:
t[:,2] = t[:, 1] + np.random.uniform(size = 1000) / t[:, 3]

In [ ]:
(np.random.uniform(size = (1000)) / t[:, 3]).shape

In [ ]:
###
cds.race_model(boundary_fun = bf.constant,
              n_samples = 100000)

In [ ]:
np.empty((100 ,10), dtype = np.float32)

In [ ]:
x = np.random.uniform(size = (20, 10))

In [ ]:
x[(x < 0.2)] = 1000

In [ ]:
t = tuple()

In [ ]:
np.random.uniform()

In [ ]:
b = ('analytic' in a)

In [ ]:
np.quantile(np.random.uniform(size = (10000,4)), q = [0.05, 0.10, 0.9, 0.95], axis = 0)

In [ ]:
np.quantile(np.random)

In [ ]:
a = np.array([False, False, True])

In [ ]:
np.invert(a)

In [ ]:
(np.array([0, 0, 0])).astype(bool) + (np.array([0, 0, 1])).astype(bool)

In [ ]:
a = np.random.uniform(size = ((3,3)))

In [ ]:
tuple(map(tuple, a))

In [ ]:
tuple(np.apply_along_axis(my_func, 0, a, key_vec))

In [ ]:
dict(zip(a[0,:], ['a' ,'b', 'c']))

In [ ]:
def my_func(x = 0, key_vec = ['a' ,'b', 'c']):
    return dict(zip(key_vec, x))

In [ ]:
my_func_init = my_func(key_vec = ['d', 'e', 'f'])

In [ ]:
key_vec = ['d', 'e', 'f']

In [ ]:
a = tuple()

In [ ]:
a = tuple(({} for i in range(10)))

In [ ]:
tuple(a)

In [ ]:
b = tuple()

In [ ]:
b+= a[0]

In [ ]:
import yaml


In [ ]:
test = yaml.load(open('config_files/config_data_generator.yaml'))

In [ ]:
b =tuple([i[1] for i in a])

In [ ]:
a = [['a', 1], ['b', 2]]

In [ ]:
[i[1] for i in a]

In [13]:
from tqdm import tqdm

In [22]:
from multiprocessing import Pool

def myfunc(a):
    return a ** 2
pbar = tqdm(total = 100)
def update():
    pbar.update

a = tuple()
for i in range(pbar.total):
    a += ((1, ), )
    
pool = Pool(4)
pool.starmap(myfunc, a, callback = update)
pool.close()
pool.join()






  0%|          | 0/100 [00:00<?, ?it/s]

TypeError: starmap() got an unexpected keyword argument 'callback'

In [18]:
a

((1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,),
 (1,))

In [ ]:
def my_fun(*args):
    print(args)

In [ ]:
my_fun((1,2,3,4))

In [8]:
import dataset_generator as dg

In [3]:
out = dg.make_dataset_r_dgp(dgp_list = ['weibull_cdf', 'angle', 'ddm', 'ornstein', 'lba'], 
                            save = False)

(1, 0.01, 20, 200, False, <function weibull_cdf at 0x7f9b9b4b6378>, True)
(0.07808740423548866, 0.8387635386775256, 0.45082497697048957, 0.6739789614476364)
({'alpha': 3.254405760554941, 'beta': 6.922865174311779},)
(1, 0.01, 20, 200, False, <function weibull_cdf at 0x7f9b9b4b6378>, True)
(0.3302378701981361, 0.7031297408339329, 0.3087315026984216, 0.49544435292264055)
({'alpha': 4.102995757885083, 'beta': 2.1270465978793807},)
(1, 0.01, 20, 200, False, <function weibull_cdf at 0x7f9b9b4b6378>, True)
(-1.147024123898614, 0.6961861485880171, 0.3111995030350717, 0.30308666007787854)
({'alpha': 1.5328508974736046, 'beta': 4.124529973947155},)
(1, 0.01, 20, 200, False, <function weibull_cdf at 0x7f9b9b4b6378>, True)
(-0.5848477293820333, 0.6077718104754508, 0.48707558811740936, 0.6190078558989981)
({'alpha': 4.820490801900434, 'beta': 5.248899741200132},)
(1, 0.01, 20, 200, False, <function weibull_cdf at 0x7f9b9b4b6378>, True)
(0.9091419631963888, 1.4471339523316935, 0.44327067973939144, 

/media/data_cifs/afengler/git_repos/nn_likelihoods/dataset_generator.py:441: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(open("/media/data_cifs/afengler/git_repos/nn_likelihoods/config_files/config_data_generator.yaml"))


0  repetition finished
(1, 0.01, 20, 200, False, <function angle at 0x7f9b9b4b6268>, False)
(0.8870838313676046, 1.338835097461862, 0.6282495837279041, 0.686855516049373)
({'theta': 0.08944831071693571},)
(1, 0.01, 20, 200, False, <function angle at 0x7f9b9b4b6268>, False)
(-1.0985270302724643, 0.7208868811707663, 0.4661756877433961, 0.18564692928514448)
({'theta': 0.4454890091243294},)
(1, 0.01, 20, 200, False, <function angle at 0x7f9b9b4b6268>, False)
(-0.5443059818929905, 0.6372222846166621, 0.5455319334917703, 0.607615796807532)
({'theta': 0.6486252030314689},)
(1, 0.01, 20, 200, False, <function angle at 0x7f9b9b4b6268>, False)
(-1.3109353947215743, 0.8497755964596148, 0.6463460563356038, 0.4995147061048415)
({'theta': 0.9214799143334544},)
(1, 0.01, 20, 200, False, <function angle at 0x7f9b9b4b6268>, False)
(-0.8372518302613181, 1.0763232933250424, 0.6311780163860072, 0.7463287302288268)
({'theta': 1.2971512634511435},)
(1, 0.01, 20, 200, False, <function angle at 0x7f9b9b4b6268

0  repetition finished
(1, 0.01, 20, 200, False, <function constant at 0x7f9b9b4a6d90>, True)
(0.6957583843460267, 1.1500203925847043, 0.3125263295817515, 0.7962106413474608)
({},)
(1, 0.01, 20, 200, False, <function constant at 0x7f9b9b4a6d90>, True)
(0.6507089761285325, 0.6327830855710886, 0.49757941628420266, 0.8366886060914392)
({},)
(1, 0.01, 20, 200, False, <function constant at 0x7f9b9b4a6d90>, True)
(1.3506124051887802, 0.9336319326881385, 0.630567175168836, 0.28124474410471945)
({},)
(1, 0.01, 20, 200, False, <function constant at 0x7f9b9b4a6d90>, True)
(0.6457451192321746, 0.5689072842253593, 0.4021772049406527, 0.588544597322028)
({},)
(1, 0.01, 20, 200, False, <function constant at 0x7f9b9b4a6d90>, True)
(1.5858380663517306, 1.1053340554059299, 0.6331729673218358, 0.37267590420448893)
({},)
(1, 0.01, 20, 200, False, <function constant at 0x7f9b9b4a6d90>, True)
(-0.833716460783966, 0.5782248267114117, 0.5620461738652094, 0.9724459215372772)
({},)
(1, 0.01, 20, 200, False, <f

0  repetition finished
(1, 0.01, 20, 200, False, <function constant at 0x7f9b9b4a6d90>, True)
(-1.4852685768161602, 1.2725246085409263, 0.5380364585080113, 0.6496807066299146, 0.07589985454944104)
({},)
(1, 0.01, 20, 200, False, <function constant at 0x7f9b9b4a6d90>, True)
(1.1211958343627073, 0.9189797691201725, 0.43319063814945896, -0.9401319523833842, 0.6748500431116403)
({},)
(1, 0.01, 20, 200, False, <function constant at 0x7f9b9b4a6d90>, True)
(-0.39263802195177133, 1.4456866330379325, 0.4889873101084497, -0.022684737579132586, 0.19248256155638055)
({},)
(1, 0.01, 20, 200, False, <function constant at 0x7f9b9b4a6d90>, True)
(0.5752767652815204, 1.0091625794471555, 0.6272010243985744, 0.24493619005031264, 0.9124517187360357)
({},)
(1, 0.01, 20, 200, False, <function constant at 0x7f9b9b4a6d90>, True)
(0.7182751407239042, 1.46704205620728, 0.4596792716389579, -0.12935372460711725, 0.39754337505403103)
({},)
(1, 0.01, 20, 200, False, <function constant at 0x7f9b9b4a6d90>, True)
(-0.

0  repetition finished
[[1.0, 2.0], [1.0, 2.0], [1.0, 2.0], [0.0, 1.0], [1.5, 3.0], [0.1, 0.2], [0.0, 1.0]]
(array([1.87251649, 1.71290645]), 1.2056061789181083, 0.957289650701594, 2.9271197909776525, 0.133783489232903)
(array([1.02219724, 1.3889768 ]), 1.1019427889135691, 0.05445388469138801, 2.004254928511592, 0.1128341580698246)
(array([1.73769024, 1.59745929]), 1.400179907841197, 0.7429003359801715, 1.766987697628817, 0.18620825380958986)
(array([1.86718401, 1.90559584]), 1.543548957918338, 0.4781452524198945, 1.5521498736121235, 0.18701877327650188)
(array([1.88376802, 1.43551962]), 1.879079843380861, 0.6451345641922686, 1.709249906839289, 0.16657589414738855)
(array([1.63609303, 1.67925282]), 1.9234614351749164, 0.604447523020211, 2.3538216427280565, 0.1524109286477972)
(array([1.44495412, 1.08499568]), 1.1461415962852497, 0.9618503452217357, 2.8705345257740422, 0.1542957738716175)
(array([1.56426834, 1.89041661]), 1.3756041871656297, 0.3087164583783968, 1.7533241529798085, 0.145

(array([1.1040941 , 1.47757865]), 1.6524720726910123, 0.4872918053708808, 2.131663976515917, 0.11617015874280956)
({},)
(200, 20.0, 0.01)
(array([1.69027119, 1.88904221]), 1.9114314804697479, 0.9244810908687926, 2.3677455897679174, 0.1887356764932308)
({},)
(200, 20.0, 0.01)
(array([1.45358086, 1.71825478]), 1.8461030491013741, 0.0005527428154799008, 2.9126533414592157, 0.19964462966838947)
({},)
(200, 20.0, 0.01)
(array([1.75381126, 1.54370986]), 1.9205481774396382, 0.5415261313704656, 1.7373504976390468, 0.10874466257486366)
({},)
(200, 20.0, 0.01)
(array([1.66691377, 1.2166232 ]), 1.567418071983681, 0.3310830331527618, 2.1964567491151703, 0.1936171308264729)
({},)
(200, 20.0, 0.01)
(array([1.43622893, 1.03189204]), 1.1825204002095768, 0.0923106802249305, 2.978526109364821, 0.10895651376516217)
({},)
(200, 20.0, 0.01)
(array([1.08824377, 1.3120423 ]), 1.801227823048729, 0.5871370368959559, 2.56472415322614, 0.10599676440401053)
({},)
(200, 20.0, 0.01)
(array([1.55984272, 1.08767924])

In [ ]:
help(dg.make_dataset_r_dgp)

In [ ]:
def zip_dict(x = [], 
             key_vec = ['a', 'b', 'c']):
    return dict(zip(key_vec, x))

In [ ]:
my_dg = dg.data_generator(file_id = 'TEST')

In [ ]:
out = my_dg.make_dataset_perturbation_experiment(save = False)

In [ ]:
a = np.zeros((100, 2))
b = ['a', 'b']

In [ ]:
a = np.apply_along_axis(zip_dict, 1,
                    a, b)

In [ ]:
out = my_dg.make_dataset_uniform(save = False)

In [ ]:
my_dg.param_grid_perturbation_experiment()

In [ ]:
param_grid = my_dg.param_grid_uniform()

In [ ]:
%%timeit -n 1 -r 1
tt = my_dg.generate_data_grid_parallel(param_grid = param_grid)

In [ ]:
my_dg.config

In [ ]:
a = {'a': 10, 'b': 10}

In [ ]:
tuple(a.values())

In [ ]:
a[:, 1] = ['a', 'a', 'a']

In [ ]:
a = np.random.choice(10, size = (1000,1))

In [25]:
for i in zip([1,2,3], [1, 2, 3], [1]):
    print(i)

(1, 1, 1)
